 ╔══<i><b>Alai-DeepLearning</b></i>════════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 9. CNN Basis**
# Section 4. 합성곱 층(Convolution Layer)의 구조

### _Objective_
1. 합성곱 층은 이전에 배운 DNN와 비슷하게, Logit을 계산하는 부분과 활성화 함수를 계산하는 부분으로 나누어집니다. <br>
2. 고전 CNN 모델을 살펴보면서, CNN의 구조에 대해 배워보도록 하겠습니다.  <br> 
  
╚═════════════════════════════════════════╝

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

<br><br>

# \[ 1. 합성곱 층의 구조 \]

----
----
 
> *합성곱 연산도 이전의 Fully Connected Layed와 마찬가지로, Bias가 존재합니다.*<br>
> *합성곱 연산 이후 활성화 함수를 넣음으로써, 비선형성을 증대시킵니다.*<br>

## 1. 합성곱 연산에 Bias 추가하기
---

* 각 필터 별로 Bias를 추가함으로써, 필터 별 Threshold을 학습할 수 있게 됩니다.<br>

### (1) 3차원 데이터 구성하기

In [0]:
in0 = np.array([
    [1,4,2,0],
    [2,3,1,1],
    [3,1,3,2],
    [4,2,1,4],
])
in1 = np.array([
    [1,7,0,2],
    [3,2,1,1],
    [0,7,5,1],
    [0,1,5,2],    
])
in2 = np.array([
    [1,5,1,5],
    [3,2,1,9],
    [4,2,2,7],
    [1,2,1,9],
])
inputs = np.stack([in0, in1, in2], axis=-1)
print("입력값의 형태 (H, W, C) :({},{},{})".format(*inputs.shape))

입력값의 형태 (H, W, C) :(4,4,3)


### (2) Filter 와 Bias 구성하기
* 필터 별로 독립적인 Bias가 존재합니다. 필터가  $(n_f,h_f,w_f,c_{in})$로 구성되어 있을 때, Bias는 $n_f$개 만큼 존재합니다.

In [0]:
filter_1 = np.array([
    [[2,0,1],[0,1,2],[1,0,2]],
    [[0,1,3],[2,1,3],[4,1,2]],
    [[3,2,1],[2,2,3],[0,0,1]]])
filter_2 = np.array([
    [[4,0,1],[0,0,4],[0,3,2]],
    [[6,1,2],[3,5,1],[2,3,2]],
    [[1,4,1],[1,3,1],[2,1,0]]])

filters = np.stack([filter_1, filter_2])
print("Filter의 형태 (N, H, W, C) : ({},{},{},{})".format(*filters.shape))

Filter의 형태 (N, H, W, C) : (2,3,3,3)


In [0]:
bias = np.array([-100,20])
print("bias의 갯수 : {}".format(len(bias)))

bias의 갯수 : 2


### (3) 합성곱 연산 진행하기

In [0]:
# 합성곱 연산 진행하기
outputs = np.zeros([2,2,2])
for i in range(2):
    for j in range(2):
#         for k in range(2):
        patch = np.expand_dims(inputs[i:i+3, j:j+3], 0)
        result = (patch * filters).reshape([2,-1]).sum(axis=1)
        outputs[i,j] = result
outputs

array([[[ 94., 118.],
        [110., 153.]],

       [[ 90., 134.],
        [117., 148.]]])

In [0]:
outputs.shape

(2, 2, 2)

In [0]:
# bias를 더해줌
outputs = outputs+bias#fix me#
outputs

array([[[ -6., 138.],
        [ 10., 173.]],

       [[-10., 154.],
        [ 17., 168.]]])

<br>
## 2. 합성곱 연산 후 활성화함수 추가하기
---

* 이전의 DNN 구조와 동일하게 합성곱을 거친 후, 활성화함수를 거침으로써,<br>
보다 복잡한 특징들을 추출할 수 있게 됩니다.

In [0]:
def relu(x):
    return np.maximum(x,0)

In [0]:
# 활성화 함수를 거침
a = relu(outputs)
a

array([[[  0., 138.],
        [ 10., 173.]],

       [[  0., 154.],
        [ 17., 168.]]])

합성곱 신경망의 한 계층은 DNN과 같이 두단계를 거칩니다.<br>
1. 합성곱 연산(with bias)
2. 활성화 함수

<br>
## 3. 합성곱 층의 Notation
---

* 이후 논문과 모델 구조를 보면서 빠르게 이해할 수 있도록, 주요 Notation을 정리하겠습니다.

$
f^{[l]} = \mbox{filter size} \\
p^{[l]} = \mbox{padding} \\
s^{[l]} = \mbox{stride} \\
k^{[l]} = \mbox{number of filters}\\
----\\
\mbox{input shape : } (n_h^{[l-1]},n_w^{[l-1]},n_c^{[l-1]}) \\
\mbox{filter shape : } ( f^{[l]}_h,f^{[l]}_w,n_c^{[l-1]},k^{[l]}) \\
\mbox{output shape : } (n_h^{[l]},n_w^{[l]},n_c^{[l]})
$

### (1) 출력 층의 크기 (output shape)

출력 층의 높이와 폭은 아래의 수식 구조를 따릅니다.

$
n_h^{[l]} = \lfloor \frac{n_h^{[l-1]}+2p^{[l]}-f_h^{[l]}}{s^{[l]}}+1\rfloor
$<br>
$
n_w^{[l]} = \lfloor \frac{n_w^{[l-1]}+2p^{[l]}-f_h^{[l]}}{s^{[l]}}+1\rfloor
$


### (2) 출력 값의 채널 수

출력 층의 채널의 수($n_c^{[l]}$)는 필터의 갯수($k^{[l]}$)와 동일합니다.

$
n_c^{[l]} = k^{[l]}
$

### (3) 파라미터의 수 

각 합성곱 층은 Filter 내 Weight와 Bias로 이루어져 있습니다.<br>
$
\mbox{#parameter} = \mbox{#weights} + \mbox{#bias} \\
\mbox{#weights} = f_h^{[l]} * f_w^{[l]} * n_c^{l-1} * k^{[l]} \\
\mbox{#bias} = n_c^{[l]}
$

<br><br>

# \[ 2. CNN 모델 분석하기 \]

----
----
 
> *고전 모델 중 하나인 LeNet-5을 각 단계별로 출력 값의 크기 및 파라미터의 수를 계산해보도록 하겠습니다.*<br>


간단한 CNN 모델의 구성을 통해 계산해보기
----
![Imgur](https://i.imgur.com/DHpS6r8.png)


각 층 별 필터의 크기, 스트라이드, 패딩은 아래와 같습니다.

| Layer | # filters | filter size | stride | padding |
| ----- | -------   | ------ | ----- | ----- |
|  C1   | 6 | (5,5) | 1 | 0 |
|  S2   | 6 | (2,2) | 2 | 0 |
|  C3   | 16 | (5,5) | 1 | 0 |
|  S4   | 16 | (2,2) | 2 | 0 |
|  C5   | 120 | (5,5) | 1 | 0 |
|  F6   | 84 | --- | --- | --- |
|  OUTPUT   | 10 | --- | --- | --- |

## 1. 각 층 별 출력 층의 크기 계산하기
----

출력층은 아래의 수식을 따릅니다.<br>
$
n_h^{[l]} = \lfloor \frac{n_h^{[l-1]}+2p^{[l]}-f_h^{[l]}}{s^{[l]}}+1\rfloor
$<br>
$
n_w^{[l]} = \lfloor \frac{n_w^{[l-1]}+2p^{[l]}-f_h^{[l]}}{s^{[l]}}+1\rfloor
$



#### 예제) 각 층 별 출력 층의 크기 계산하기 

| Layer | FEATURE MAP SIZE |
| ----- | -------   | 
| INPUT | (32,32,1) |
|  C1   | ? |
|  S2   | ? |
|  C3   | ? |
|  S4   | ? |
|  C5   | ? |
|  F6   | ? |
|OUTPUT | ? |

#### 정답 :

| Layer | FEATURE MAP SIZE |
| ----- | -------   | 
| INPUT | (32,32,1) |
|  C1   | (28,28,6) |
|  S2   | (14,14,6) |
|  C3   | (10,10,16) |
|  S4   | (5,5,16) |
|  C5   | (1,1,120)|
|  F6   | (84,) |
|  OUTPUT   | (10,) |

## 2. 각 층 별 파라미터의 수 계산하기
----

각 합성곱 층은 Filter 내 Weight와 Bias로 이루어져 있습니다.<br>
$
\mbox{#parameter} = \mbox{#weights} + \mbox{#bias} \\
\mbox{#weights} = f_h^{[l]} * f_w^{[l]} * n_c^{l-1} * k^{[l]} \\
\mbox{#bias} = n_c^{[l]}
$<br>
풀링층은 별도로 학습하는 파라미터가 없습니다.

#### Caution
* sub-sampling은 Max-Pooling으로 생각하고 계산해 주세요.

#### 예제) 각 층 별 출력 층의 크기 계산하기 

| Layer | # Parameter |
| ----- | -------   | 
| INPUT | 0 |
|  C1   | $(5*5*1*6) + 6 = 156 $ |
|  S2   | ? |
|  C3   | ? |
|  S4   | ? |
|  C5   | ? |
|  F6   | ? |
|OUTPUT | ?|

#### 정답 :

| Layer | # Parameter |
| ----- | -------   | 
| INPUT | 0 |
|  C1   | $(5*5*1*6) + 6 = 156 $ |
|  S2   | $0$ |
|  C3   | $(5*5*6*16) + 16 = 2,416 $ |
|  S4   | $0$ |
|  C5   | $(5*5*16*120)+120 = 48,120$ |
|  F6   | $120*84 + 84 = 10,164 $ |
|OUTPUT | $84*10 + 10 = 850$ |

# \[ 3.간단한 Convolution Neural Network 만들기 \]

----
----
 
> *자신만의 Convolution Neural Network 을 생성해보고 결과를 확인해 봅니다.*<br>


In [0]:
import tensorflow as tf 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [0]:
xs_flat = tf.placeholder(tf.float32, [None, 784])#fix me#
xs = tf.reshape(xs_flat, [-1, 28, 28, 1])#fix me#
ys = tf.placeholder(tf.float32, [None, 10])#fix me#
ys_cls = tf.one_hot(ys, 10)#fix me#

with tf.name_scope('layer_1'): #fix me# : scope name is layer_1
    kernel_init = tf.random_normal([3,3,1,100], stddev=0.1)#fix me # random normal 
    kernel = tf.Variable(kerner_init, 'kernel')# fix me #
           
    bias_init = tf.zeros([100])
    bias = tf.Variable(bias_init, 'bias')

    layer = tf.nn.conv2d(xs, kernel, [3,3,100,64], 'SAME') # convolution and add bias        
    layer = tf.nn.relu(layer)#fix me# # apply activation function 
    
with tf.name_scope('layer_2'): # : scope name is layer_2
    kernel_init = tf.random_normal([3,3,100,64], stddev=0.1)#fix me # random normal 
    kernel = tf.Variable(kernel_init, 'kernel')# fix me # 
    bias_init = tf.zeros([64])# fix me # 
    bias=tf.Variable(bias_init, 'bias')# fix me # 
    
    layer = tf.nn.conv2d(layer, kernel, [1,3,3,1], 'SAME')# fix me # 
    layer = tf.nn.relu(layer)# fix me # apply activation funciton 
    
with tf.name_scope('FC'):
    h, w, ch = map(int, tf.get_shape(layer)[1:])
    flat_layer = tf.reshape(layer, [-1, h*w*ch])#fix me# flatten layer 
    logits = tf.layers.Dense(10)(flat_layer)#fully connected layer # 

    logits_cls = tf.argmax(logits, axis=1)#onehot to number#
    
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=ys_onehot,logits=logits)#fix me #
acc = tf.metrics.accuracy(labels=ys_cls, predictions=logits_cls)#fix me #
lr = 0.0001
train_op = tf.train.tf.train.AdamOptimizer(0.001).minimize(loss)#fix me# Adam optimizer

TypeError: ignored

In [0]:
sess = tf.Session()
init = tf.group(#fix me #) # global , local intializer
sess.run(init)

for i in range(30000):
    batch_xs, batch_ys = mnist.train.next_batch(120)
    _, loss_ = sess.run(#fix me#)
    
    if i % 1000 ==0:
        val_loss, val_acc = sess.run(#fix me#)
        print(val_loss, val_acc[1])

3.4300566 0.0966
0.2663954 0.5106
0.16920345 0.65826666
0.122966625 0.7353
0.09534456 0.78292
0.08291871 0.81523335
0.0698219 0.8387714


KeyboardInterrupt: ignored